# Giới thiệu TensorFlow và các thao tác cơ bản
## MaSSP 2017, Computer Science

__Chuẩn bị: Xếp Châu__

## 0. Giới thiệu TensorFlow
<img style="width: 320px; height: 320px; float: left;" src="../../images/Intro_TensorFlow/tensorflow_logo.png" />

<li>__TensorFlow__ là một thư viện mở được phát triển bởi các nhà nghiên cứu và kĩ sư trong Google Brain Team. 
<li>Mục đích ban đầu nhằm thực hiện nghiên cứu trong Machine Learning và Deep neural networks, nhưng hệ thống có thể áp dụng cho nhiều lĩnh vực khác. 

<img style="width: 350px; height: 260px; float: right;" src="../../images/Intro_TensorFlow/dataflow_graph.png" />
 
<li> __TensorFlow__ thực hiện các tính toán trong một __session__ </li>
<li> các tính toán được biểu diễn dưới dạng __graph__</li>
<li> mỗi phép tính (__operation__) là một node </li>
<li> dữ liệu, biến được biễu diễn dưới dạng __tensor__</li>

Khi thực hiện tính toán trong graph, graph phải được chạy trong một __session__.

__Session__ sẽ đưa các __operation__ trong __graph__ vào CPU hoặc GPU để thực hiện chúng.

<li> __Tensor__ là một _typed multi-dimensional array_</li>

Ví dụ, một nhóm các hình ảnh có thể được biểu diễn bằng một 4-D array tensor gồm các số thực. Bốn chiều của array này là $[batch, height, width, channels]$. $Batch$ tương ứng với số hình ảnh, và 3 giá trị còn lại ta đã nắm rõ từ lab xử lý ảnh, chúng tương ứng với chiều cao, độ rộng của một ảnh, và số kênh của một hình ảnh.

<img style="width: 450px; height: 270px; float: left;" src="../../images/Intro_TensorFlow/operations.png" />

<li> Một __operation__ có thể nhận $0$ hoặc nhiều hơn $0$ Tensor, thực hiện một số tính toán, và tạo ra $0$ hoặc nhiều hơn $0$ __tensor__.</li>

## 1. Xây dựng graph đơn giản nhất
Trong phần này, chúng ta sẽ học cách truy cập __graph__ trong TensorFlow, thêm vào graph một số __operations__ đơn giản, và tạo ra một __session__ để thực hiện các operations trong graph.

Đầu tiên chúng ta import tensorflow và viết tắt nó là <i>tf</i>.

In [2]:
import tensorflow as tf

Lúc này, TensorFlow cung cấp sẵn một __graph__, ta có thể dùng lệnh $get\_default\_graph()$ để thu được graph có sẵn này.

In [3]:
graph = tf.get_default_graph()
graph

>Từ output trên ta thấy được graph là một đối tượng của class Graph, class này được định nghĩa tại đây: https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/framework/ops.py

Như đã đề cập, mỗi node trong graph là một operation. Hàm $get\_operations()$ sẽ trả về một _list các operations_ đang có trong graph. Định nghĩa của hàm này có thể được tìm thấy trong class Graph.

<img src="../../images/Intro_TensorFlow/graph_get_operations.png" />

In [4]:
operations = graph.get_operations()
operations

[]

Lúc này graph chưa có operation nào cả. Ta hãy thử khởi tạo một hằng số $x$ với giá trị $1.0$ xem graph thay đổi thế nào.

In [5]:
x = tf.constant(1.0)

Việc khởi tạo một hằng số này là một operation, cũng chính là một node trong graph.

In [6]:
operations = graph.get_operations()
operations

>Operation class cũng được định nghĩa tại đây: https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/framework/ops.py

Ta có thể dùng $node\_def$ để thấy rõ hơn cấu trúc của operation này.

In [7]:
operations[0].node_def

name: "Const"
op: "Const"
attr {
  key: "dtype"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "value"
  value {
    tensor {
      dtype: DT_FLOAT
      tensor_shape {
      }
      float_val: 1.0
    }
  }
}

Lưu ý, chúng ta có thể đánh giá input $x$ theo cách thông thường, nhưng sẽ chỉ thu được những thông tin sau.

In [9]:
print(x)

Tensor("Const:0", shape=(), dtype=float32)


__Checkpoint 1__: Bạn hãy đoán xem kết quả của lệnh "print" trên thu được nhờ hàm nào trong https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/framework/ops.py?  

Kết quả cho thấy input $x$ là một hằng số, 32-bit float tensor không có chiều, nhưng không cho ta biết giá trị của hằng số này là gì. Để đánh giá input $x$ và nhận được giá trị số của nó, chúng ta cần đến một __session__.

In [11]:
sess = tf.Session()    # khởi tạo session
sess.run(x)            # thực hiện operation x, tf.constant(1.0)

1.0

In [12]:
sess.close()           # đóng session

Lưu ý, ngoài $Session()$, chúng ta sẽ sử dụng $InteractiveSession()$ rất nhiều trong chương trình. Với $InteractiveSession$, chúng ta không cần viết $sess.run(x)$, mà có thể viết $x.eval()$ để thực hiện một operation.

In [13]:
sess = tf.InteractiveSession()
x.eval()

1.0

Hãy nhớ đóng session lại cuối chương trình!

In [14]:
sess.close()

Chúng ta vừa học cách tạo ra một hằng số trong TensorFlow, cách nhận biết các operations trong __graph__, và dùng __session__ để đánh giá một operation. 

Tuy nhiên graph hiện tại chưa thực hiện một phép tính toán đúng nghĩa nào. Hãy cùng xây dựng một _neuron_ có thể thực hiện được một phép tính toán đơn giản trong phần sau.

## 2. Xây dựng TensorFlow neuron
<img style="float: left" src="../../images/Intro_TensorFlow/a_simple_neuron.png" />
Trong phần này, chúng ta sẽ mô phỏng lại một neuron đơn giản. Neuron này chỉ có 2 tham số cần tìm và cả 2 đều là số thực, đó là weight $w$ và bias $b$. Neuron sẽ nhận một giá trị input $x$, và xuất ra một giá trị output $y$ theo công thức 

$output = weight * input + bias$, hay $y = w*x + b$

Neuron sẽ bắt đầu với giá trị weight $w$ và bias $b$ bất kì. Chúng ta sẽ cho neuron giá trị của input $x$, và output $y\_correct$ mong muốn tương ứng với input $x$ này.

Với giá trị weight $w$, bias $b$ ban đầu và input $x$, neuron áp dụng công thức tính ra output $y$, và tất nhiên sẽ có sai số với output mong muốn $y\_correct$.

Để giúp neuron điều chỉnh giá trị weight $w$ và bias $b$ cho phù hợp (sao cho output $y$ của neuron sẽ gần hơn với $y\_correct$), chúng ta sẽ cung cấp cho neuron một hàm tính sai số __cost function__.

Sai số càng lớn khi output $y$ của neuron càng xa với $y\_correct$, và neuron sẽ thay đổi weight $w$ và bias $b$ nhiều hơn khi sai số càng lớn.

Do weight $w$ và bias $b$ sẽ biến thiên tùy vào sai số trong quá trình học, ta sẽ sử dụng biến TensorFlow ($tf.Variable$ class) cho chúng.

Code của $tf.Variable$ class: https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/ops/variables.py.

Đầu tiên, khởi tạo biến weight $w$ ban đầu, giả sử có giá trị ban đầu là $0.8$.

In [15]:
w = tf.Variable(0.8)
w

__Checkpoint 2__: So với khởi tạo một hằng số, việc khởi tạo một biến thêm vào graph bao nhiêu operations? Hãy in ra tên của những operations đó và đoán xem nhiệm vụ của chúng là gì?

_Gợi ý: sử dụng $op.name$ để biết được tên của operation $op$._

In [ ]:
# code 


Tương tự, ta khởi tạo biến bias $b$ với giá trị ban đầu bất kì, giả sử là $0.5$.

In [16]:
b = tf.Variable(0.5)
b

Hãy sử dụng lại code từ Checkpoint 2 để biết các operations thêm vào graph từ việc khởi tạo bias $b$.

In [ ]:
# code


Với input $x$ $1.0$ từ phần 1 và weight $w$, bias $b$ vừa khởi tạo, ta đã sẵn sàng thêm vào graph một operation tính toán thật sự khi tính output $y$.

In [17]:
y = w * x + b

Hãy viết code để biết được __2 operation mới nhất__ nào vừa được thêm vào graph.

In [20]:
# code


__Checkpoint 3__: Bạn có thể đoán được đoạn code có mục đich gì không?

In [ ]:
for op in graph.get_operations():
    print("Operation name: {}".format(op.name))
    for op_input in op.inputs:
        print("\tOp input: {}".format(op_input))

Hãy thử tính kết quả của phép nhân trên bằng $InteractiveSession$.

In [24]:
sess = tf.InteractiveSession()
y.eval()

FailedPreconditionError: Attempting to use uninitialized value Variable_1
	 [[Node: Variable_1/read = Identity[T=DT_FLOAT, _class=["loc:@Variable_1"], _device="/job:localhost/replica:0/task:0/cpu:0"](Variable_1)]]

Caused by op 'Variable_1/read', defined at:
  File "C:\Users\vumin\Anaconda3\envs\tensorflow\lib\runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\vumin\Anaconda3\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\vumin\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "C:\Users\vumin\Anaconda3\envs\tensorflow\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\vumin\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "C:\Users\vumin\Anaconda3\envs\tensorflow\lib\site-packages\zm

Thay vì nhận được kết quả, chúng ta lại gặp lỗi này:

<span style="color: red">"FailedPreconditionError (see above for traceback): Attempting to use uninitialized value Variable"</span>

Phép tính $y$ cần có các biến weight $w$ và bias $b$. Mặc dù chúng ta đã khởi tạo các biến này bằng $tf.Variable$, giá trị của chúng vẫn chưa được gán trong session hiện tại. Hàm $tf.global\_variables\_initializer()$ sẽ làm nhiệm vụ init tất cả các biến trong graph (trong trường hợp này chỉ có hai biến là weight $w$ và bias $b$).

In [25]:
init = tf.global_variables_initializer()

Hàm $tf.global\_variables\_initializer()$ này cũng trả về một operation, nên đừng quên dùng session để thực hiện operation này!

In [28]:
sess.run(init)

Cuối cùng ta có thể tính output $y$.

In [29]:
y.eval()

1.3

Đây là kết quả tính hợp lý của $output = weight * input + bias = 0.8 * 1.0 + 0.5 = 1.3$.

Qua phần này, ta thấy rằng bản thân mỗi operation đều chứa thông tin về các input của nó (cũng là những operation khác trong graph). 

Tưởng tượng với số lượng operation tăng lên, sẽ rất khó nhớ được mối quan hệ giữa các operation.

[TensorBoard graph visualization](https://www.tensorflow.org/how_tos/graph_viz/) là một công cụ giúp ta hình dung graph trong TensorFlow.

## 3. Dùng TensorBoard để phân tích graph

__TensorBoard__ là một công cụ hữu ích giúp thể hiện mối quan hệ của các operation trong graph cũng như xem thông tin của từng operation. Ngoài ra TensorBoard còn nhiều chức năng khác như vẽ đồ thị, hình ảnh..., chúng ta sẽ tìm hiểu song song với các bài thực hành tiếp theo.

Đầu tiên, ta cần đến $FileWriter$, cùng với tên của thư mục ta sẽ lưu dữ liệu của graph vào (ví dụ "Lab2") và graph chúng ta cần biểu diễn.

In [ ]:
file_writer = tf.summary.FileWriter("Lab2", sess.graph)

Bây giờ bạn hãy tạm rời Jupyter Notebook và dùng command shell (sau khi đã "activate tensorflow") để chạy TensorBoard với lệnh _"tensorboard --logdir=Lab2"_.

In [ ]:
#(tensorflow) D:\MaSSP\track_2>tensorboard --logdir=Lab2
#Starting TensorBoard b'39' on port 6006
#(You can navigate to http://10.0.0.5:6006)

Thư mục "Lab2" sẽ chứa thông tin thu lại từ graph. Command còn cho ta biết một đường link có thể được truy cập từ trình duyệt web, nơi ta có thể nhìn thấy graph:

<img style="width: 970px; height: 400px;" src='../../images/Intro_TensorFlow/log_simple_graph.png' />

Ta nhận thấy rằng __operation__ có tên khác với tên biến trong Python (ta không thấy có chữ output $y$, input $x$ trong graph, thay vào đó là $Const\_1$, $Variable\_2$ hoặc tương tự).

Tuy nhiên ta có thể đặt tên các operation này để phân biệt chúng dễ dàng hơn trong graph.

In [ ]:
x = tf.constant(1.0, name="input")
w = tf.Variable(0.8, name="weight")
b = tf.Variable(0.5, name="bias")
y = tf.add(tf.mul(w, x), b, name="output")

Hãy viết lại graph lần nữa. 

(Lưu ý khi làm việc với Jupyter notebook các bạn nên xóa thư mục "Lab2" và khởi động lại kernel trước khi thực hiện tính toán một lần nữa, nếu không graph sẽ chứa nhiều bản sao của cùng một operation.)

In [ ]:
tf.summary.FileWriter("Lab2", sess.graph)

...khởi động lại TensorBoard với "Lab2", và refresh trang web.

In [ ]:
#(tensorflow) D:\MaSSP\track_2>ls
#... Lab2_Tutorial1

#(tensorflow) D:\MaSSP\track_2>tensorboard --logdir=Lab2_Tutorial1
#Starting TensorBoard b'39' on port 6006
#(You can navigate to http://10.0.0.5:6006)

Lúc này, tên các operation sẽ xuất hiện trong graph tương tự như sau.

<img src='../../images/Intro_TensorFlow/named_vars.png' />

Ngoài cách đặt tên biến, ta còn có thể sử dụng hàm $tf.name\_scope()$ để nhóm các operation lại với nhau. Việc nhóm các operation tương tự giúp biểu diễn cách graph phức tạp dễ dàng hơn và hiểu cách thành phần của quá trình tính toán rõ ràng hơn.

In [ ]:
with tf.name_scope("a_simple_neuron") as scope:
    x = tf.constant(1.0)
    w = tf.Variable(0.8, name="weight")
    b = tf.Variable(0.5, name="bias")
    y = tf.add(tf.mul(w, x), b, name="output")

Và đây là kết quả thu được trong TensorBoard, các operation được nhóm lại trong scope "a_simple_neuron" và có thể được triển khai hoặc thu gọn bằng cách nhấn vào dấu "+" hoặc "-" tương ứng.

__Checkpoint 4__: Hãy khởi động lại TensorBoard để thấy scope "a_simple_neuron".
<img src='../../images/Intro_TensorFlow/a_simple_neuron_scope.png' />

## 4. Train neuron - Linear Regression
Quay lại với neuron đơn giản của chúng ta với chỉ 2 thông số là weight $w$ và bias $b$ và các thông số này có giá trị ban đầu lần lượt là là $0.8$ và $0.5$. Với input $x$ là $1.0$, neuron tính giá trị output $y$ là $1.3$. 

Tuy nhiên, giả sử $y\_correct$ chúng ta mong muốn lại là $1.6$, thay vì $1.3$, ta cần giúp neuron biết: 
1. giá trị output $y$ nó tính được là chưa chuẩn (lý do là vì weight $w$ và bias $b$ chưa chuẩn!), và 
2. chỉ cho neuron biết cách điều chỉnh weight $w$ và bias $b$ để giảm sai số giữa output $y$ neuron tính được và output thực tế $y\_correct$.

In [ ]:
# Khởi tạo output mong muốn y_expect
y_correct = tf.constant(1.6, name="correct_output")

### 4.1 Hàm sai số
Ta cần định nghĩa một hàm sai số để tính được độ lệch của output $y$ và $correct\_output$. Trong trường hợp này chúng đều là 2 số thực, và ta sẽ sử dụng hàm sai số least squares:

$$cost = (correct\_output - output)^2$$

In [ ]:
y_correct = tf.constant(1.6)
with tf.name_scope("cost_function") as scope:
    cost = (y_correct - y)**2

Ta có thể đánh giá $cost$ với interactive session hiện tại.

In [ ]:
cost.eval()

__Checkpoint 5__: Kết quả $cost$ này có hợp lý không?

### 4.2 Điều chỉnh tham số
Từ giá trị hiện tại của $cost$, ta cần điều chỉnh weight $w$ và bias $b$. Hiển nhiên nếu sai số $cost$ càng lớn thì ta nên thay đổi càng nhiều weight $w$ và bias $b$.

Trong bài giảng, ta đã biết đến cách dùng thuật toán Gradient Descent để điều chỉnh tham số dựa vào đạo hàm của $cost$. 

TensorFlow cung cấp hàm $GradientDescentOptimizer()$ cho việc này, lấy $learning\_rate$ là giá trị thể hiện tốc độ học nhanh hay chậm của quá trình học. Nếu tốc độ học quá lớn, khả năng cao ta sẽ không đạt được giá trị nhỏ nhất của sai số. Tốc độ học quá chậm lại dẫn đến việc quá trình học quá lâu, qua nhiều bước mà sai số vẫn lớn.

Ta sẽ thử tốc độ học là $0.025$. Và định nghĩa một bước học là quá trình điều chỉnh weight $w$ và bias $b$ dựa vào sai số $cost$.
>Gradient Descent Optimizer: https://www.tensorflow.org/api_docs/python/tf/train/GradientDescentOptimizer

In [ ]:
with tf.name_scope("train_step") as scope:
    train_step = tf.train.GradientDescentOptimizer(0.025).minimize(cost)

Hàm $tf.summary.scalar$ giúp ta quan sát được các biến thay đổi thế nào trong quá trình học.

In [ ]:
file_writer = tf.summary.FileWriter("Lab2", sess.graph)
sess.run(tf.global_variables_initializer())

# Tạo các đồ thị thể hiện sự biến thiên của weight, bias, cost, và output
summary_weight = tf.summary.scalar('weight', w)
summary_bias = tf.summary.scalar('bias', b)
summary_cost = tf.summary.scalar('cost', cost)
summary_y = tf.summary.scalar('output', y)

Lúc này ta đã sẵn sàng thực hiện quá trình "train" neuron. Cụ thể ta sẽ gọi bước $train\_step$ 100 lần, và trong mỗi bước ghi lại sự biến thiên của các biến nhờ $FileWriter$.

Để tiện theo dõi, phần code cần thiết để chạy chương trình này đã được viết trong file $Intro\_TensorFlow\_mini\_regression.py$.

In [ ]:
# Thực hiện quá trình học
for i in range(100):
    # Tính và log các giá trị hiện tại của weight, bias, cost, và output
    summaries = sess.run([summary_weight, summary_bias, summary_cost, summary_y])
    for summary in summaries:
        file_writer.add_summary(summary, i)
        
    # Gọi bước học tiếp theo
    sess.run(train_step)

__Checkpoint 6__: Hãy sử dụng TensorBoard để trả lời các câu hỏi sau:
- Giá trị cuối cùng của weight, bias, và cost?
- Giá trị cuối cùng của output có hợp lý không?
- Đến khoảng bước nào thì cost giảm không đáng kể?
- Tại sao sau bước này thì giá trị của weight, bias, cost, và output không thay đổi nhiều?

<img src='../../images/Intro_TensorFlow/a_simple_neuron_one_data_point.png' />

__Checkpoint 7__: Hãy thay đổi learning_rate và lặp lại các bước trên, quan sát và nhận xét các đồ thị thu được
- learning_rate = 0.001
- learning_rate = 0.00001
- learning_rate = 0.4
- learning_rate = 0.5

### 4.3 Kết luận
Chúng ta vừa xây dựng một neuron đơn giản bằng TensorFlow với 2 thông số weight $w$ và bias $b$. Với một cặp giá trị $(input, correct\_output)$ cho sẵn, neuron đã biết cách học và tự điều chỉnh weight $w$ và bias $b$ sao cho output $y$ mà neuron tính được càng gần với output thật $y\_expected$. Số bước học và sai số của output $y$ phụ thuộc nhiều vào tốc độ học, hàm sai số, và optimizer.

Tiếp theo, chúng ta sẽ cho neuron học dựa trên nhiều cặp giá trị $(input, correct\_output)$ khác nhau, thay vì một cặp $(0.8, 1.6)$ như trên (vô số nghiệm thỏa mãn cho hệ phương trình 2 ẩn, 1 phương trình). Mặt khác mỗi input có thể không phải là một số thực (thể hiện 1 thuộc tính), mà có nhiều thuộc tính và input khi đó sẽ được biếu diễn bởi 1 vector. Ví dụ:

<img src="../../images/Intro_TensorFlow/housing_cost.png" />
$$ x = 
    \begin{bmatrix}
        [2104 & 5 & 1 & 45] \\
        [1416 & 3 & 2 & 40] \\
        [1534 & 3 & 2 & 30] \\
         [852 & 2 & 1 & 36] \\
         ...
    \end{bmatrix}
; \ \ \  y = 
    \begin{bmatrix}
         460 \\
         232 \\
         315 \\
         178 \\
         ...
    \end{bmatrix}         
$$

__Bài tập__: Hãy tham khảo phần code trong file "Intro_TensorFlow_regression.py" để biết cách khởi tạo $x$ là một array thay vì là số thực như trong "Intro_TensorFlow_mini_regression.py".

## Hỏi đáp